In [5]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

symbols = [ "H","Be", "H"]
coordinates = np.array([ 0.0,0.0,-1.29,0.0,0.0,0.0, 0.0,0.0,1.29])
basis_set = "sto-3g"


H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    active_electrons=2,
    active_orbitals=3
)

In [6]:
singles, doubles = qchem.excitations(electrons=2, orbitals=qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 8


In [7]:
hf = qchem.hf_state(2, qubits)

In [8]:


num_theta = len(singles) + len(doubles)

def circuit_VQE(params,expectations ):
    qml.AllSinglesDoubles(
        weights = params,
        wires = range(qubits),
        hf_state = hf,
        singles = expectations[0],
        doubles = expectations[1])
    return qml.expval(H)

In [9]:
dev = qml.device("default.qubit", wires=qubits)
cost_fn = qml.QNode(circuit_VQE, dev)

circuit_gradient = qml.grad(cost_fn, argnum=0)
n=len(doubles)+ len(singles)
params = [0.0] * n
expectations = [singles,doubles]
grads = circuit_gradient(params, expectations)

for i in range(n):
    print(f"Gradient: {grads[i]}")

Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: -0.06718200764707273
Gradient: 0.0
Gradient: 0.0
Gradient: -0.0671820076470818


In [10]:
single_select = [singles[i] for i in range(len(singles)) if abs(grads[i])>2e-10]

In [11]:
n_s = len(singles)
double_select = [doubles[i] for i in range(len(doubles)) if abs(grads[n_s+i])>1e-4]

In [12]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time
cost_fn = qml.QNode(circuit_VQE, dev)
stepsize = 0.4
opt = qml.GradientDescentOptimizer(stepsize=stepsize)
params = np.zeros(len(double_select + single_select), requires_grad=True)

gates_select =  [single_select,double_select]

for n in range(20):
    t1 = time.time()
    params, energy = opt.step_and_cost(cost_fn, params, expectations=gates_select)
    t2 = time.time()
    print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))

n = 0,  E = -14.77344237 H, t = 0.50 s
n = 1,  E = -14.77678285 H, t = 0.40 s
n = 2,  E = -14.77919511 H, t = 0.22 s
n = 3,  E = -14.78093260 H, t = 0.20 s
n = 4,  E = -14.78218183 H, t = 0.21 s
n = 5,  E = -14.78307892 H, t = 0.18 s
n = 6,  E = -14.78372260 H, t = 0.29 s
n = 7,  E = -14.78418420 H, t = 0.81 s
n = 8,  E = -14.78451511 H, t = 0.20 s
n = 9,  E = -14.78475228 H, t = 0.20 s
n = 10,  E = -14.78492224 H, t = 0.29 s
n = 11,  E = -14.78504403 H, t = 0.11 s
n = 12,  E = -14.78513130 H, t = 0.19 s
n = 13,  E = -14.78519384 H, t = 0.51 s
n = 14,  E = -14.78523865 H, t = 0.41 s
n = 15,  E = -14.78527075 H, t = 0.20 s
n = 16,  E = -14.78529376 H, t = 0.21 s
n = 17,  E = -14.78531025 H, t = 0.22 s
n = 18,  E = -14.78532207 H, t = 0.17 s
n = 19,  E = -14.78533054 H, t = 0.40 s


In [1]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
# pylint: enable=line-too-long

import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit_nature.settings import settings
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
settings.dict_aux_operators = False
#from  qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver

/tmp/ipykernel_30/3308095054.py:2: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms import (GroundStateEigensolver,
/tmp/ipykernel_30/3308095054.py:2: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms import (GroundStateEigensolver,
/opt/conda/envs/pennylane/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3553: ListAuxOpsDeprecationWarning: List-based `aux_operators` are deprecated as of version 0.3.0 and support for them will be removed no sooner than 3 mont

In [ ]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["Be", [0.0, 0.0, 0.0]],
            ["H", [-dist, 0.0, 0.0]],
            ["H", [dist, 0.0, 0.0]]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    

    driver = ElectronicStructureMoleculeDriver(
         molecule=molecule,
         basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)
    transformer = ActiveSpaceTransformer(
        num_electrons=4, # Number of electrons in our active space
        num_molecular_orbitals=5, # Number of orbitals in our active space
    )
    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [transformer, FreezeCoreTransformer(freeze_core=True)])
    
    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles
    mapper = ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops[0]  # Set Hamiltonian
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)
    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [17]:
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result
from qiskit.providers.fake_provider import FakeGuadalupe, FakeLagos
#backend = FakeGuadalupe()
backend = FakeLagos()
#backend = BasicAer.get_backend("statevector_simulator")
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

# pylint: disable=undefined-loop-variable


dist = 1.3
(qubit_op, num_particles, num_spin_orbitals,
                            problem, converter) = get_qubit_op(dist)
result = exact_solver(problem,converter)
exact_energies.append(result.total_energies[0].real)
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
var_form = UCCSD(converter,
                    num_particles,
                    num_spin_orbitals,
                    initial_state=init_state)
vqe = VQE(var_form, optimizer, quantum_instance=backend)
print('1')
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
print('2')
vqe_result = problem.interpret(vqe_calc).total_energies[0].real
vqe_energies.append(vqe_result)
print(f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Exact Energy: {exact_energies[-1]:.5f}")

print("All energies have been calculated")

1
2
Interatomic Distance: 1.3 VQE Result: -13.81260 Exact Energy: -15.56744
All energies have been calculated


In [4]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["Be", [0.0, 0.0, 0.0]],
            ["H", [-dist, 0.0, 0.0]],
            ["H", [dist, 0.0, 0.0]]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    

    driver = ElectronicStructureMoleculeDriver(
         molecule=molecule,
         basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)
    transformer = ActiveSpaceTransformer(
        num_electrons=2, # Number of electrons in our active space
        num_molecular_orbitals=3, # Number of orbitals in our active space
    )
    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [transformer])
    
    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles
    mapper = ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops[0]  # Set Hamiltonian
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)
    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [5]:
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result
from qiskit.providers.fake_provider import FakeGuadalupe, FakeLagos
#backend = FakeGuadalupe()
backend = FakeLagos()
#backend = BasicAer.get_backend("statevector_simulator")
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

# pylint: disable=undefined-loop-variable


dist = 1.3
(qubit_op, num_particles, num_spin_orbitals,
                            problem, converter) = get_qubit_op(dist)
result = exact_solver(problem,converter)
exact_energies.append(result.total_energies[0].real)
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
var_form = UCCSD(converter,
                    num_particles,
                    num_spin_orbitals,
                    initial_state=init_state)
vqe = VQE(var_form, optimizer, quantum_instance=backend)
print('1')
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
print('2')
vqe_result = problem.interpret(vqe_calc).total_energies[0].real
vqe_energies.append(vqe_result)
print(f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Exact Energy: {exact_energies[-1]:.5f}")

print("All energies have been calculated")

1
2
Interatomic Distance: 1.3 VQE Result: -14.88059 Exact Energy: -15.56185
All energies have been calculated
